設定ファイルを読み込む

- ``sandbox/config.toml``
- 質問をカテゴリー型に変換するための設定
- ``[questions]``: 質問の内容を取得するための設定
- ``[choices]``: 質問の回答をカテゴリー型に変換するための設定
- ``choices``は使いまわすことがあるので、``questions``と1対1に対応させず、別々のセクションにした
  - 例: ``regional`` / ``subregional`` / ``good_poor`` / ``agree_disgree`` など

In [ ]:
from pathlib import Path
import pandas as pd
import tomllib

``tomllib``を使って設定ファイルを読み込む

In [ ]:
f_cfg = "../sandbox/config.toml"
fname = Path(f_cfg)
with fname.open("rb") as f:
    config = tomllib.load(f)

キーを確認する

In [ ]:
config.keys()

In [ ]:
nd = config.get("defined", "Not defined")
volumes = config.get("volumes", "Not Defined")
questions = config.get("questions", "Not Defined")
choices = config.get("choices", "Not Defined")
o = config.get("options", "Not Defined")
options = pd.DataFrame(o)

In [ ]:
nd

In [ ]:
volumes.keys()

In [ ]:
questions.keys()

In [ ]:
choices.keys()

質問の概要オプションをデータフレームに変換する

In [ ]:
options["name"].tolist()
options.query("type == 'categorical'")["name"].to_list()
options.query("type == 'categorical'")["description"].to_list()
options.query("type == 'categorical'")[["name", "title"]].to_dict().keys()

## ``config``モジュールの動作確認

- ``Config``クラス、``Data``クラスは``__init__.py``で呼び出していて、トップレベル（``ti.Config``、``ti.Data``）で使うことができる

## ``Config``クラス

- ``load_from``で設定ファイルを指定する
  - 現在は``../sandbox``にある``config.toml``
- インスタンスを作成しただけでは設定は読み込まれていない
  - ``Config``クラスは（無駄に）``pydantic``の``BaseModel``を継承している
  - 正直``dataclasses``でよかった気がしている
- ``Config.load``を実行すると、設定ファイルの内容を読み込むことできる
  - このときにメンバー変数（``config`` / ``categories`` / ``optins``）も設定される

In [ ]:
import titanite as ti
f_cfg = "../sandbox/config.toml"
c = ti.Config(load_from=f_cfg)
c

``load_from``で指定したファイルの内容を読み込む

In [ ]:
#c.load_categories()
#c.load_options()
c.load()
#c.categories

設定を確認する

- 設定内容は``self.config``でアクセスできる
- ``self.config``は辞書型

In [ ]:
type(c.config)
c.config.keys()
#c.config.get("volumes", "Not defined")
#c.config.get("questions", "Not defined")
#c.config.get("choices", "Not defined").items()
#c.config.get("options", "Not defined")
#c.config.get("overview", "Not defined")

- ``categorical()`` を ``load_categories()``で置き換える
- ``self.categories``でアクセスできるようにした

In [ ]:
c.categorical() == c.categories

- 質問に関係するオプション一式はデータフレームにした
- ``load_options()``で読み込み＆変換ができる
- ``self.options``でアクセスできるようにした

In [ ]:
c.options

## ``Data``クラス

- ``Data``クラスも``pydantic``の``BaseModel``を継承している
- ``Config``クラスとは独立したクラスとして作成した
- 前処理済のデータを読み込み、要素のカテゴリー化を一括しておこなう

In [ ]:
import titanite as ti
f_cfg = "../sandbox/config.toml"
f_csv = "../data/test_data/prepared_data.csv"
c = ti.Config(load_from=f_cfg)
t = ti.Data(read_from=f_csv, load_from=f_cfg)

データを読み込む

In [ ]:
data = t.read()
data.info()

In [ ]:
ti.config.CATEGORICAL_HEADERS
ti.config.NUMERICAL_HEADERS

総当たりするカラム名のペアを列挙する

In [ ]:
headers = list(data.columns)
t.matches(columns=headers)

クロス集計